In [2]:
# Import libraries and functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import keras
import tensorflow as tf
import warnings
import os
from keras import layers
from keras import models
from keras import optimizers
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
warnings.filterwarnings('ignore')

In [3]:
url = 'Datasets/2012_2019_PlatteRiverWeir_features_merged_all.csv'

df = pd.read_csv(url)

In [4]:
# df

In [5]:
imglist = []
for path in os.listdir('Datasets/images/')+df[' Filename']:
    imglist.append(os.path.join('Datasets/images/', path))

In [6]:
df2 = df
df2['images'] = imglist
df2 = df2.sample(frac=1).reset_index(drop=True)
# df2

In [7]:
df2[["year", "month", "day"]] = df["SensorTime"].str.split("-", expand = True)

In [8]:
df2[["day", "time"]] = df2["day"].str.split(" ", expand = True)

In [9]:
# df2

In [10]:
dfGroup = df2.groupby(df2['year'])

In [11]:
df2012 = dfGroup.get_group("2012")
df2013 = dfGroup.get_group("2013")
df2014 = dfGroup.get_group("2014")
df2015 = dfGroup.get_group("2015")
df2016 = dfGroup.get_group("2016")
df2017 = dfGroup.get_group("2017")
df2018 = dfGroup.get_group("2018")
df2019 = dfGroup.get_group("2019")


In [12]:
dfTrain = pd.concat([df2012, df2013, df2014, df2018, df2019])
# dfTrain

In [13]:
dfTest = pd.concat([df2015, df2017])
# dfTest

In [14]:
dfVal = df2016
# dfVal

In [15]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu',input_shape=(512,512,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='linear'))

In [16]:
model.compile(optimizer='adam', metrics=['mse'], loss=[tf.keras.metrics.MeanSquaredError()])

In [17]:
datagen = ImageDataGenerator()
train_data = datagen.flow_from_dataframe(
    dfTrain,
    directory='Datasets/images/',
    x_col=' Filename',
    y_col=' Discharge',
    target_size=(512,512),
    batch_size=64,
    class_mode='raw',
    validate_filenames=True
)


val_data = datagen.flow_from_dataframe(
    dfVal,
    directory='Datasets/images/',
    x_col=' Filename',
    y_col=' Discharge',
    target_size=(512,512),
    batch_size=64,
    class_mode='raw',
    validate_filenames=True
)


Found 23462 validated image filenames.
Found 5533 validated image filenames.


In [18]:
history = model.fit(train_data, epochs=5, validation_data=val_data)

Epoch 1/5


ValueError: in user code:

    File "C:\Users\alex_\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\alex_\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\alex_\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\alex_\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 997, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "C:\Users\alex_\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\optimizer_v2.py", line 579, in minimize
        return self.apply_gradie